# Instagram Media Downloader

Automates browser inspection method: monitors network requests and downloads media from Instagram CDN.

In [ ]:
!pip install playwright requests
!playwright install chromium

In [ ]:
import asyncio
import os
import requests
from pathlib import Path
from datetime import datetime
from playwright.async_api import async_playwright

In [ ]:
# Configuration
INSTAGRAM_USERNAME = "your_username"
INSTAGRAM_PASSWORD = "your_password"
DOWNLOAD_FOLDER = "instagram_downloads"
HEADLESS = False

Path(DOWNLOAD_FOLDER).mkdir(exist_ok=True)

In [ ]:
class MediaCollector:
    def __init__(self):
        self.media_urls = set()
        self.downloaded_urls = set()
        
    def handle_response(self, response):
        url = response.url
        content_type = response.headers.get('content-type', '')
        
        if 'image' in content_type and 'cdninstagram.com' in url:
            self.media_urls.add(('image', url))
        elif 'video' in content_type or '.mp4' in url:
            self.media_urls.add(('video', url))
    
    def download_media(self, media_type, url, folder):
        if url in self.downloaded_urls:
            return None
            
        try:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            ext = '.mp4' if media_type == 'video' else '.jpg'
            url_hash = abs(hash(url)) % 10000
            filename = f"{media_type}_{timestamp}_{url_hash}{ext}"
            filepath = os.path.join(folder, filename)
            
            response = requests.get(url, stream=True, timeout=30)
            if response.status_code == 200:
                with open(filepath, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                
                self.downloaded_urls.add(url)
                print(f"✓ {filename}")
                return filepath
        except Exception as e:
            print(f"✗ Error: {str(e)}")
        return None
    
    def download_all(self, folder):
        print(f"\nDownloading {len(self.media_urls)} items...")
        return [self.download_media(t, u, folder) for t, u in self.media_urls if self.download_media(t, u, folder)]

In [ ]:
class InstagramDownloader:
    def __init__(self, username, password, headless=False):
        self.username = username
        self.password = password
        self.headless = headless
        self.collector = MediaCollector()
        
    async def login(self, page):
        await page.wait_for_selector('input[name="username"]', timeout=10000)
        await page.fill('input[name="username"]', self.username)
        await page.fill('input[name="password"]', self.password)
        await page.click('button[type="submit"]')
        await asyncio.sleep(5)
        
        # Dismiss popups
        for text in ["Not now", "Not Now"]:
            try:
                btn = page.locator(f'button:has-text("{text}")')
                if await btn.count() > 0:
                    await btn.first.click()
                    await asyncio.sleep(2)
            except:
                pass
        print("✓ Logged in")
    
    async def browse_profile(self, page, url, num_posts):
        await page.goto(url)
        await asyncio.sleep(3)
        
        # Click first post
        await page.locator('article a').first.click()
        await asyncio.sleep(3)
        
        for i in range(num_posts):
            print(f"Post {i+1}/{num_posts}")
            await asyncio.sleep(3)
            
            # Handle carousel
            try:
                carousel_btn = page.locator('button[aria-label="Next"]').first
                for _ in range(10):
                    if await carousel_btn.is_visible():
                        await carousel_btn.click()
                        await asyncio.sleep(2)
                    else:
                        break
            except:
                pass
            
            # Next post
            if i < num_posts - 1:
                try:
                    next_btn = page.locator('a:has-text("Next")')
                    if await next_btn.count() > 0:
                        await next_btn.first.click()
                        await asyncio.sleep(3)
                    else:
                        break
                except:
                    break
    
    async def run(self, profile_url, num_posts=10):
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=self.headless)
            context = await browser.new_context(
                viewport={'width': 375, 'height': 812},
                user_agent='Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15'
            )
            page = await context.new_page()
            page.on('response', self.collector.handle_response)
            
            try:
                await page.goto('https://www.instagram.com/')
                await asyncio.sleep(3)
                await self.login(page)
                await self.browse_profile(page, profile_url, num_posts)
                
                downloaded = self.collector.download_all(DOWNLOAD_FOLDER)
                print(f"\n✓ Downloaded {len(downloaded)} files to '{DOWNLOAD_FOLDER}'")
            finally:
                await browser.close()

In [ ]:
# Run downloader
downloader = InstagramDownloader(
    username=INSTAGRAM_USERNAME,
    password=INSTAGRAM_PASSWORD,
    headless=HEADLESS
)

await downloader.run(
    profile_url="https://www.instagram.com/natgeo/",
    num_posts=5
)